# Robert Chatterton

#### DS 4400 Machine Learning and Data Mining I Final Project

<hr />

## Authorship Attribution on Classical Works and More

`Outline:`

    - Clean and pull data
        - Read in texts from NLTK
        - Choose specific ones (5 authors?)

    - Prepare data
        - N-grams
        - Bag of Words
        - word2Vec? if I can
        - Get style
            - sentence length
            - word choice
            - sentiment
            

    - Models:
        - Logistic Regression
        - Random Forest
        - Naive Bayes
        - NN (LSTM or Feed forward?)

In [15]:
# imports
import numpy as np
import nltk
import matplotlib.pyplot as plt

# Data Cleaning

In [16]:
nltk.download('gutenberg')
nltk.download('punkt')

[nltk_data] Downloading package gutenberg to /home/robbie/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!
[nltk_data] Downloading package punkt to /home/robbie/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [37]:
minimum_sentence_length = 5

# authors and works to investigate
all_works = {
    'austen': {
        'austen-persuasion.txt': [],
        'austen-sense.txt': []
    },
    'chesterson': {
        'chesterton-ball.txt': [],
        'chesterton-brown.txt': [],
        'chesterton-thursday.txt': []
    },
    'shakespeare': {
        'shakespeare-caesar.txt': [],
        'shakespeare-hamlet.txt': [],
        'shakespeare-macbeth.txt': []
    }
}

# for each author, grab each of their works, separated into sentences
for author, works in all_works.items():
    total_sents = 0
    for work in works.keys():
        raw = nltk.corpus.gutenberg.raw(work)
        
        # fix up text
        text = raw.replace('."', '".').replace('?"', '"?').replace('!"', '"!')
        text = text.replace('--', ' ').replace('. . .', '').replace('_', '')
        text = text.replace('\n', ' ').replace('  ', ' ').replace('  ', ' ')

        # tokenize into sentences
        sents = nltk.tokenize.sent_tokenize(text)

        # remove short sentences
        sents = [sent for sent in sents if len(sent) > minimum_sentence_length]
        all_works[author][work] = sents
        total_sents += len(sents)
        print(f'{work} has {len(sents)} sentences')
    print(f'{author} has {total_sents} in the dataset')

austen-persuasion.txt has 3599 sentences
austen-sense.txt has 5134 sentences
austen has 8733 in the dataset
chesterton-ball.txt has 4591 sentences
chesterton-brown.txt has 3706 sentences
chesterton-thursday.txt has 3554 sentences
chesterson has 11851 in the dataset
shakespeare-caesar.txt has 1423 sentences
shakespeare-hamlet.txt has 2048 sentences
shakespeare-macbeth.txt has 1284 sentences
shakespeare has 4755 in the dataset


In [36]:
for i in range(5):
    print(all_works['austen']['austen-emma.txt'][i])
    print()

[Emma by Jane Austen 1816] VOLUME I CHAPTER I Emma Woodhouse, handsome, clever, and rich, with a comfortable home and happy disposition, seemed to unite some of the best blessings of existence; and had lived nearly twenty-one years in the world with very little to distress or vex her.

She was the youngest of the two daughters of a most affectionate, indulgent father; and had, in consequence of her sister's marriage, been mistress of his house from a very early period.

Her mother had died too long ago for her to have more than an indistinct remembrance of her caresses; and her place had been supplied by an excellent woman as governess, who had fallen little short of a mother in affection.

Sixteen years had Miss Taylor been in Mr. Woodhouse's family, less as a governess than a friend, very fond of both daughters, but particularly of Emma.

Between them it was more the intimacy of sisters.

